<a href="https://colab.research.google.com/github/JayBlaine/WSBAI/blob/main/WSBAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Regression model

In [ ]:
#!apt-get -qq install -y libfluidsynth1
#!pip install numpy
#!pip install pandas
#!pip install yfinance
#!pip install tensorflow
#!pip install scikit-learn
#!pip install praw
#!pip install asyncpraw
#!pip install datetime

     |████████████████████████████████| 61kB 2.5MB/s 
     |████████████████████████████████| 256kB 7.0MB/s 


In [ ]:
import pandas as pd
import numpy as np
import pandas_datareader as web
import datetime as dt
import csv
import os

def main():
    stonk = "GME"
    flairs = {"Gain", "Loss", "YOLO"}
    start = dt.datetime(2021, 1, 1)
    end = dt.datetime.today()

    data = web.DataReader(stonk,'stooq', start, end).reset_index()
    data.drop('Open', 1, inplace=True)
    data.drop('High', 1, inplace=True)
    data.drop('Low', 1, inplace=True)

    if not os.path.exists('data'):
        os.makedirs('data')

    data = data.to_csv(r'./data/stonkdata.csv', index=False, header=True)
    
if __name__ == '__main__':
    main()

In [ ]:
# WSB data set read
# DO NOT RUN AFTER LIVE SCRAPE STARTS
# SERIOUSLY, YOU WILL RUIN DATASET ONCE LIVE SCRAPING BEGINS

import pandas as pd
import csv
import datetime as dt

def gmeChecker(title, body):
  gmeKeys = ['gme', 'gamestop', 'dfv', 'deep fucking value', 'citadel', 'ken griffin', 'keith', 'gill', 'roaring kitty', 'if he\'s in']
  title = title.strip().lower()
  body = body.strip().lower()
  for key in gmeKeys:
    # body and title
    if key in title or key in body:
      return True

  return False

def dateChecker(date1):

  dtTemp = dt.datetime.strptime(date1, '%Y-%m-%d %H:%M:%S')
  actDate = dtTemp.date()

  return actDate

  


# date, reddit total posts, gme total posts, ratio of gme to total, 
# avg score of reddit posts, average score of gme posts, avg comments total, avg comments gme
redditData = [] # 8 values, 2d list
tempCSVHolder = []
byDate = {}

with open('reddit_wsb.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:

      if line_count > 0:
        tempTitle = row[0]
        tempScore = row[1]
        tempCommNum = row[4]
        tempBody = row[6]
        tempDate = row[7]
        tempPass = [tempTitle, tempScore, tempCommNum, tempBody, tempDate]
        #tempCSVHolder.append(tempPass)

        #tester = gmeChecker(tempTitle, tempBody)
        date1 = dateChecker(tempDate)

        if date1 not in byDate.keys():
          byDate[date1] = []
          byDate[date1].append(tempPass)
          # print(byDate[date1][0][4])
        
        else:
          byDate[date1].append(tempPass)

      line_count+=1

redditPrelim = []

for key in byDate.keys():

  if key.year < 2021: #Gets rid of rogue row at 9/2020
      continue

  i=0
  gmeCnt = 0
  gmeCommTotal = 0
  gmeUpTotal = 0
  while i < len(byDate[key]):

    
    gmeCheck = gmeChecker(byDate[key][i][0], byDate[key][i][3])
    if gmeCheck:
      gmeCnt+=1
      gmeCommTotal+=int(byDate[key][i][2])
      gmeUpTotal+=int(byDate[key][i][1])

    #TODO
    i+=1
  if i > 0 and gmeCnt > 0: # both posts and gme posts
    redditPrelim.append([key, i, gmeCnt, gmeCnt/i, gmeUpTotal/gmeCnt, gmeCommTotal/gmeCnt])
  elif i > 0: # no gme posts
    redditPrelim.append([key, i, gmeCnt, gmeCnt/i, 0, 0])
  else: # no posts
    continue

  #print(gmeCnt)
  #print(i)
#print(redditPrelim)


with open('wsb_clean.csv', mode='w') as write_file:
    writer = csv.writer(write_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    i=0
    #TODO
    writer.writerow(['Date', 'Total Posts', 'GME Count', '% GME', 'Avg GME Upvotes', 'Avg GME Comments'])
    while i < len(redditPrelim):
      writer.writerow([redditPrelim[i][0], redditPrelim[i][1], redditPrelim[i][2], redditPrelim[i][3], redditPrelim[i][4], redditPrelim[i][5]])
      i+=1



In [ ]:
# REDDIT DATA SCRAPE
import csv
import pandas as pd
import asyncpraw
import nest_asyncio
nest_asyncio.apply()

async def main():
  reddit = asyncpraw.Reddit(client_id='ORNE1J4ncT_tGg', client_secret= 'sa-2SPPkOY-w_kMIBSeAostYFg804Q', user_agent='WSBAI:Python/urllib:v0.0.1 (by u/WSBAI_00123)', 
                            username="WSBAI_00123", password="lint$h@d0w,./")
  wsb_sub = await reddit.subreddit('WallStreetBets')

  
  i = 0
  print(reddit.read_only)
  posts = [] # posts that are good from keywords
  keywords = ['GME', 'gamestop', 'diamond', 'diamond hands', 'ape', 'apes']
  postIds = {}

  # TODO: 

  # -Keyword list for relevant posts/comments
  # -handle emojis
  # -post id dictionary for posts already read through
  # -possible CV of reddit posts if jpg/png
  # -store into csv file

  async for submission in wsb_sub.new(limit=20):
    test = submission.downvote()
    # submission.id
    # submission.created_utc   CHECK FOR LESS THAN DAY OLD

    # submission.title
    # submission.num_comments
    # submission.upvote_ratio
    # submission.link_flair_text
    # num text keywords
    # num comment keywords


    # submission.url           POSSIBLE IMG CHECKING
    
    # submission.score   ----MAY NOT NEED----
    
    
    #---STUFF TO LOOP THROUGH TO COUNT NUM OF KEYWORDS--- comments, selftext
    #submission.selftext
    #comments = await submission.comments()
    #for comment in comments:
    #  print(comment.body)
    #  print(comment.score)

    #posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
  #posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])




if __name__ == '__main__':
  loop = asyncio.get_event_loop() 
  loop.run_until_complete(main())


False

https://v.redd.it/aim61y5xz9p61

https://i.redd.it/omvy6g8qz9p61.jpg


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: coroutine 'VotableMixin.downvote' was never awaited



Retail Investors deserve the right to trade in a free & equal market, without manipulation. 

Time to move on from the SEC & contact FINRA regarding market manipulation. See [Finra.org](https://Finra.org) \- 

Dark Pool transactions are still kept accountable, which means there are audit records for all trades. FINRA is the keeper of these records.

[https://www.finra.org/rules-guidance/notices/18-25](https://www.finra.org/rules-guidance/notices/18-25)

**Regulatory Notice 18-25** 

FINRA is issuing this *Notice* to remind Alternative Trading Systems (ATSs) of their supervision obligations.1 As registered broker-dealers and FINRA members, ATSs—like other broker-dealer trading platforms—are required to maintain supervisory systems that are reasonably designed to achieve compliance with applicable securities laws and regulations, and with applicable FINRA rules, including, for example, rules on disruptive or manipulative quoting and trading activity.

[https://www.finra.org/rules-guidan

/usr/lib/python3.7/asyncio/tasks.py:249: RuntimeWarning: coroutine 'VotableMixin.downvote' was never awaited
  result = coro.send(None)


In [ ]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(1)

1
